In [67]:
import sys
import os


from VAE_Module_2 import *

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchsummary import summary
from torchmetrics.regression import R2Score, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError


In [68]:
DATA_PATH = '../Data/'

electric_df = pd.read_csv(DATA_PATH + 'electric_df_clear.csv')
water_df = pd.read_csv(DATA_PATH + 'water_df_clear.csv')

In [69]:
print(electric_df.head())
print()
print(water_df.head())

      0     1     2     3     4     5     6     7     8     9  ...    18  \
0  1.23  1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  ...  1.77   
1  1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  ...  1.68   
2  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  ...  1.20   
3  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  ...  1.42   
4  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  1.39  ...  1.88   

     19    20    21    22    23    24    25    26    27  
0  1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  
1  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  
2  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  
3  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  1.68  
4  1.67  1.24  1.39  1.67  1.60  1.26  1.41  1.68  1.59  

[5 rows x 28 columns]

     0    1    2    3    4    5    6    7    8    9  ...   18   19   20   21  \
0   30  120  210  410   32  184  180  260   35  145  ...  188   95   46  139   
1  120  210

In [70]:
electric_rbscaler = MinMaxScaler().fit(electric_df)
water_rbscaler = MinMaxScaler().fit(water_df)

with open('electric_min_max_scaler.pkl', 'wb') as f:
    pickle.dump(electric_rbscaler, f)

with open('water_min_max_scaler.pkl', 'wb') as f:
    pickle.dump(water_rbscaler, f)

In [71]:
electric_df_scaled = electric_rbscaler.transform(electric_df)

water_df_scaled = water_rbscaler.transform(water_df)

electric_df = pd.DataFrame(electric_df_scaled, columns = electric_df.columns)
water_df = pd.DataFrame(water_df_scaled, columns = water_df.columns)

In [72]:
EPOCH = 100000
BATCH_SIZE = 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [73]:
electric_trainDS = AutoEncoderDataset(electric_df)
water_trainDS = AutoEncoderDataset(water_df)

electric_trainDL = DataLoader(electric_trainDS, batch_size = BATCH_SIZE)
water_trainDL = DataLoader(water_trainDS, batch_size = BATCH_SIZE)

In [76]:
input_size = 28
hidden_dim = 128
latent_dim = 64
n_layers = 3
dropout = 0.2
bidirectional = True



vae_model = VAEModel(input_size = input_size, hidden_dim = hidden_dim,
                     latent_dim = latent_dim, n_layers = n_layers,
                     dropout = dropout, bidirectional = bidirectional)

optimizer = optim.Adam(vae_model.parameters(), lr = LR)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 100, verbose = True)


c:\Users\KDP-2\anaconda3\envs\Project_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [77]:
vae_loss = training(vae_model, water_trainDL,
                    optimizer, EPOCH, scheduler, DEVICE)

[1 / 100000]
- TRAIN VAE LOSS : 26.336433693132502
[2 / 100000]
- TRAIN VAE LOSS : 26.129816035218273
[3 / 100000]
- TRAIN VAE LOSS : 26.117392793086402
[4 / 100000]
- TRAIN VAE LOSS : 26.10603884200678


KeyboardInterrupt: 